<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 825ms/step - loss: 0.6779 - accuracy: 0.5353 - val_loss: 0.6729 - val_accuracy: 0.5024
Epoch 2/10
26/26 [==============================] - 21s 796ms/step - loss: 0.6502 - accuracy: 0.6144 - val_loss: 0.6330 - val_accuracy: 0.8634
Epoch 3/10
26/26 [==============================] - 20s 785ms/step - loss: 0.6155 - accuracy: 0.7287 - val_loss: 0.5821 - val_accuracy: 0.8780
Epoch 4/10
26/26 [==============================] - 21s 796ms/step - loss: 0.5524 - accuracy: 0.7859 - val_loss: 0.4979 - val_accuracy: 0.8732
Epoch 5/10
26/26 [==============================] - 21s 801ms/step - loss: 0.5002 - accuracy: 0.7749 - val_loss: 0.4278 - val_accuracy: 0.8293
Epoch 6/10
26/26 [==============================] - 20s 788ms/step - loss: 0.4430 - accuracy: 0.8224 - val_loss: 0.4761 - val_accuracy: 0.7707
Epoch 7/10
26/26 [==============================] - 20s 781ms/step - loss: 0.3781 - accuracy: 0.8491 - val_loss: 0.2819 - val_accuracy: 0.9268

In [13]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 1328), started 0:05:52 ago. (Use '!kill 1328' to kill it.)

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.66.h5
26/26 - 22s - loss: 0.6813 - accuracy: 0.5499 - val_loss: 0.6638 - val_accuracy: 0.6098
Epoch 2/5

Epoch 00002: saving model to weights.02-0.70.h5
26/26 - 21s - loss: 0.6364 - accuracy: 0.6727 - val_loss: 0.6961 - val_accuracy: 0.4878
Epoch 3/5

Epoch 00003: saving model to weights.03-0.56.h5
26/26 - 21s - loss: 0.5957 - accuracy: 0.7019 - val_loss: 0.5591 - val_accuracy: 0.7463
Epoch 4/5

Epoch 00004: saving model to weights.04-0.52.h5
26/26 - 21s - loss: 0.5375 - accuracy: 0.7482 - val_loss: 0.5205 - val_accuracy: 0.7268
Epoch 5/5

Epoch 00005: saving model to weights.05-0.48.h5
26/26 - 20s - loss: 0.4911 - accuracy: 0.7762 - val_loss: 0.4785 - val_accuracy: 0.8000


In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 21s - loss: 0.6698 - accuracy: 0.6302 - val_loss: 0.7158 - val_accuracy: 0.4341
Epoch 2/50
26/26 - 20s - loss: 0.6483 - accuracy: 0.6472 - val_loss: 0.6562 - val_accuracy: 0.5268
Epoch 3/50
26/26 - 21s - loss: 0.6179 - accuracy: 0.7080 - val_loss: 0.6013 - val_accuracy: 0.7463
Epoch 4/50
26/26 - 20s - loss: 0.5609 - accuracy: 0.7591 - val_loss: 0.5194 - val_accuracy: 0.8439
Epoch 5/50
26/26 - 23s - loss: 0.5162 - accuracy: 0.7859 - val_loss: 0.4504 - val_accuracy: 0.8780
Epoch 6/50
26/26 - 22s - loss: 0.4743 - accuracy: 0.7956 - val_loss: 0.4124 - val_accuracy: 0.8976
Epoch 7/50
26/26 - 22s - loss: 0.3718 - accuracy: 0.8698 - val_loss: 0.3188 - val_accuracy: 0.8976
Epoch 8/50
26/26 - 22s - loss: 0.3377 - accuracy: 0.8856 - val_loss: 0.2444 - val_accuracy: 0.9415
Epoch 9/50
26/26 - 20s - loss: 0.2462 - accuracy: 0.9258 - val_loss: 0.2386 - val_accuracy: 0.9220
Epoch 10/50
26/26 - 20s - loss: 0.2201 - accuracy: 0.9282 - val_loss: 0.1596 - val_accuracy: 0.9610
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 22s 859ms/step - loss: 0.6568 - accuracy: 0.5985 - val_loss: 0.6436 - val_accuracy: 0.6293
Epoch 2/5
26/26 [==============================] - 21s 794ms/step - loss: 0.6076 - accuracy: 0.6886 - val_loss: 0.7855 - val_accuracy: 0.4390
Epoch 3/5
26/26 [==============================] - 21s 804ms/step - loss: 0.5482 - accuracy: 0.7445 - val_loss: 0.6399 - val_accuracy: 0.5463
Epoch 4/5
26/26 [==============================] - 20s 780ms/step - loss: 0.4767 - accuracy: 0.7883 - val_loss: 0.4215 - val_accuracy: 0.8878
Epoch 5/5
26/26 [==============================] - 21s 792ms/step - loss: 0.3957 - accuracy: 0.8625 - val_loss: 0.3922 - val_accuracy: 0.8683


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.598540,0.657320,0.629268,0.643635
1,1,0.688565,0.609258,0.439024,0.785497
2,2,0.744526,0.547584,0.546342,0.639940
3,3,0.788321,0.476524,0.887805,0.421487
4,4,0.862530,0.396357,0.868293,0.392223


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 22s 835ms/step - loss: 0.6682 - accuracy: 0.5949 - val_loss: 0.7043 - val_accuracy: 0.4780

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 787ms/step - loss: 0.6343 - accuracy: 0.6691 - val_loss: 0.6296 - val_accuracy: 0.7659

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 777ms/step - loss: 0.6193 - accuracy: 0.7324 - val_loss: 0.6238 - val_accuracy: 0.7707

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 783ms/step - loss: 0.6104 - accuracy: 0.7433 - val_loss: 0.6245 - val_accuracy: 0.6927

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 782ms/step - loss: 0.6

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 835ms/step - loss: 0.6603 - accuracy: 0.6168 - val_loss: 0.6194 - val_accuracy: 0.7317
Epoch 2/50
26/26 [==============================] - 21s 795ms/step - loss: 0.5921 - accuracy: 0.7141 - val_loss: 0.5513 - val_accuracy: 0.7268
Epoch 3/50
26/26 [==============================] - 20s 781ms/step - loss: 0.5372 - accuracy: 0.7457 - val_loss: 0.4883 - val_accuracy: 0.7561
Epoch 4/50
26/26 [==============================] - 21s 804ms/step - loss: 0.4763 - accuracy: 0.7895 - val_loss: 0.4404 - val_accuracy: 0.7902
Epoch 5/50
26/26 [==============================] - 20s 784ms/step - loss: 0.4475 - accuracy: 0.8005 - val_loss: 0.3978 - val_accuracy: 0.8195
Epoch 6/50
25/26 [===========================>..] - ETA: 0s - loss: 0.4044 - accuracy: 0.8338
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 20s 777ms/step - loss: 0.4080 - accuracy: 0.8309 - val_loss: 0.4784 - 

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 6207), started 0:17:36 ago. (Use '!kill 6207' to kill it.)